# XGBoost Model

In [1]:
import pandas as pd
from xgboost import XGBRegressor, callback
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
from tqdm import tqdm
from datetime import datetime
import pytz
import json
import joblib 
import os
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v9.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v9.csv")

In [3]:
# Function to analyze missing values
def analyze_missing_values(df):
    # Count and percentage of missing values per column
    missing_values = df.isna().sum()
    missing_percentage = (missing_values / len(df)) * 100
    missing_summary = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})
    
    # Display summary of missing values
    print(missing_summary[missing_summary['Missing Values'] > 0])
    
# Analyzing missing values in both datasets
print("Analysis of challenge_set_updated:")
analyze_missing_values(challenge_set_updated)

print("\nAnalysis of submission_set_updated:")
analyze_missing_values(submission_set_updated)

Analysis of challenge_set_updated:
                               Missing Values  Percentage
track_variation_ARR_100                 23528    6.375927
track_variation_DEP_100                 54966   14.895410
track_variation_ENR                     50848   13.779460
average_vertical_rate_ARR_100           23728    6.430126
average_vertical_rate_DEP_100           55412   15.016273
average_vertical_rate_ENR               50852   13.780544
average_airspeed_ARR_100                23996    6.502752
average_airspeed_DEP_100                55471   15.032262
average_airspeed_ENR                    50852   13.780544
groundspeed_ARR_100                     23725    6.429313
groundspeed_DEP_100                     55409   15.015460
groundspeed_ENR                         50850   13.780002
wind_distance_ARR_100                   23528    6.375927
wind_distance_DEP_100                   54966   14.895410
wind_distance_ENR                       50848   13.779460
average_temperature_ARR_100          

In [4]:
# Function to drop columns with more than 16% missing values, except for 'tow' in the submission set
def drop_columns_above_threshold(df, threshold=16, preserve_columns=None):
    if preserve_columns is None:
        preserve_columns = []
    
    missing_percentage = df.isna().mean() * 100
    cols_to_keep = missing_percentage[missing_percentage <= threshold].index.tolist()
    
    # Ensure columns in preserve_columns are kept even if they exceed the threshold
    cols_to_keep.extend([col for col in preserve_columns if col in df.columns])
    
    df = df[cols_to_keep]
    return df

# Applying the function to challenge_set_updated
challenge_set_updated = drop_columns_above_threshold(challenge_set_updated)

# Applying the function to submission_set_updated, keeping 'tow'
submission_set_updated = drop_columns_above_threshold(submission_set_updated, preserve_columns=['tow'])

In [5]:
# Function to handle infinities and large values
def clean_infinities(df):
    """
    Replace infinite values with NaN in the dataframe to handle large or infinite values.
    """
    return df.replace([np.inf, -np.inf], np.nan)

# Function to impute missing values using XGBoost
def xgboost_imputation(df, exclude_columns=None):
    """
    Impute missing values using XGBoost, using only the columns that do not have missing values,
    excluding the specified columns.
    
    Parameters:
    df (pd.DataFrame): The dataframe to impute.
    exclude_columns (list): List of columns to exclude from the model (e.g., 'tow').
    
    Returns:
    pd.DataFrame: Dataframe with imputed values.
    """
    df = df.copy()  # Create a copy of the dataframe to avoid modifying the original

    # Clean infinite values in the dataframe by replacing them with NaN
    df = clean_infinities(df)

    # Exclude specified columns from the imputation process
    if exclude_columns is None:
        exclude_columns = []

    # Ensure 'tow' is in the exclude columns, regardless of its state
    if 'tow' not in exclude_columns:
        exclude_columns.append('tow')

    # Select numeric columns that are not in the exclude list
    numeric_columns = df.select_dtypes(include=[np.number]).drop(columns=exclude_columns, errors='ignore')

    # Identify columns with missing values
    cols_with_missing = numeric_columns.columns[numeric_columns.isnull().any()]

    # Loop through each column that has missing values with a progress bar using tqdm
    for col in tqdm(cols_with_missing, desc="Imputing columns"):
        # Print the column being processed
        print(f"Processing column: {col}")
        
        # Separate rows with and without missing values in the target column
        missing_rows = numeric_columns[col].isnull()
        non_missing_rows = ~missing_rows

        # Find columns that do not have missing values both in non-missing and missing rows
        non_missing_features = numeric_columns.loc[non_missing_rows].dropna(axis=1).columns
        missing_features = numeric_columns.loc[missing_rows, non_missing_features].dropna(axis=1).columns

        # Ensure both training and missing feature sets are aligned and non-empty
        consistent_features = non_missing_features.intersection(missing_features)

        if consistent_features.empty:
            print(f"Skipping column {col}: No consistent features between training and prediction.")
            continue

        # Use only the consistent columns without missing values for training
        X_train = df.loc[non_missing_rows, consistent_features]
        y_train = df.loc[non_missing_rows, col]

        # Verifica se há NaNs nas features antes de treinar o modelo
        if X_train.isnull().values.any() or y_train.isnull().values.any():
            print(f"Skipping column {col}: NaNs found in training data.")
            continue

        # Treina o modelo de XGBoost
        model = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
        model.fit(X_train, y_train)

        # Verifica as colunas para as linhas com valores faltantes
        X_missing = df.loc[missing_rows, consistent_features]

        # Verifica se X_missing contém NaNs ou se as colunas de treinamento e previsão estão consistentes
        if X_missing.isnull().values.any() or X_missing.empty:
            print(f"Skipping prediction for column {col}: X_missing contains NaNs or no data available.")
            continue

        # Realiza a previsão
        df.loc[missing_rows, col] = model.predict(X_missing)

    return df

# Applying the XGBoost imputation on challenge_set_updated
challenge_set_updated = xgboost_imputation(challenge_set_updated, exclude_columns=['tow'])
# challenge_set_updated.to_csv("./data/challenge_set_updated_imputed.csv", index=False)

# Applying the XGBoost imputation on submission_set_updated
submission_set_updated = xgboost_imputation(submission_set_updated, exclude_columns=['tow'])
# submission_set_updated.to_csv("./data/submission_set_updated_imputed.csv", index=False)

Imputing columns:   0%|          | 0/27 [00:00<?, ?it/s]

Processing column: track_variation_ARR_100


Imputing columns:   4%|▎         | 1/27 [00:30<13:08, 30.31s/it]

Processing column: track_variation_DEP_100


Imputing columns:   7%|▋         | 2/27 [00:59<12:27, 29.89s/it]

...

Imputing columns:  93%|█████████▎| 25/27 [08:55<00:41, 20.69s/it]

Processing column: flown_distance_ENR


Imputing columns:  96%|█████████▋| 26/27 [09:17<00:21, 21.17s/it]

Processing column: specific_energy


Imputing columns: 100%|██████████| 27/27 [09:40<00:00, 21.50s/it]


In [6]:
# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']

n_jobs = os.cpu_count() - 1

In [7]:
# Split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the training data into training and validation sets for early stopping
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

# Define the best parameters provided
best_params = {
    'subsample': 1.0,
    'reg_lambda': 0.46415888336127775,
    'reg_alpha': 0.166810053720005,
    'min_child_weight': 4,
    'max_depth': 13,
    'learning_rate': 0.008,
    'gamma': 0.4444444444444444,
    'colsample_bytree': 0.6
}

# Initialize the XGBoost model with the provided best parameters
best_model = XGBRegressor(
    **best_params,
    objective='reg:squarederror',
    random_state=42,
    n_estimators=10_000_000,  # Set a high value to allow early stopping to find the best n_estimators
    n_jobs=n_jobs,
    eval_metric="rmse",  # Set eval_metric in the constructor
    early_stopping_rounds=20,  # Set early_stopping_rounds in the constructor
    # enable_categorical=True  # Enable categorical feature handling
)

# Train the model on the training data with early stopping using the validation set
best_model.fit(X_train, y_train, verbose=50, eval_set=[(X_val, y_val)])

# Update best_params with the best number of estimators found during early stopping
best_params['n_estimators'] = best_model.best_iteration + 1  # +1 because best_iteration is zero-indexed

# Evaluate the final model on the test set
y_pred = best_model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Best Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")
print(f"Updated best_params: {best_params}")

[0]	validation_0-rmse:53006.36066
[50]	validation_0-rmse:35616.51712
[100]	validation_0-rmse:24020.16086
[150]	validation_0-rmse:16325.07587
[200]	validation_0-rmse:11257.27182
[250]	validation_0-rmse:7976.95443
[300]	validation_0-rmse:5926.18963
[350]	validation_0-rmse:4700.24745
[400]	validation_0-rmse:4010.22660
[450]	validation_0-rmse:3641.43447
[500]	validation_0-rmse:3448.85114
[550]	validation_0-rmse:3347.26972
[600]	validation_0-rmse:3288.98414
[650]	validation_0-rmse:3252.93973
[700]	validation_0-rmse:3230.11886
[750]	validation_0-rmse:3214.24538
[800]	validation_0-rmse:3198.83707
[850]	validation_0-rmse:3186.31054
[900]	validation_0-rmse:3175.65963
[950]	validation_0-rmse:3166.53343
[1000]	validation_0-rmse:3158.86946
[1050]	validation_0-rmse:3151.40440
[1100]	validation_0-rmse:3145.03319
[1150]	validation_0-rmse:3139.48192
[1200]	validation_0-rmse:3134.68302
[1250]	validation_0-rmse:3130.61947
[1300]	validation_0-rmse:3126.87198
[1350]	validation_0-rmse:3123.26288
[1400]	val

In [8]:
# Save R², RMSE, and hyperparameters
results = {
    'R2': float(r2),
    'RMSE': float(rmse),
    'Best Parameters': {key: (int(value) if isinstance(value, np.integer) else float(value)
                              if isinstance(value, np.floating) else value)
                        for key, value in best_params.items()}
}

# Set timezone to São Paulo (UTC-3)
saopaulo_tz = pytz.timezone('America/Sao_Paulo')
timestamp = datetime.now(saopaulo_tz).strftime('%Y%m%d_%H%M%S')

# Define logs directory, and create them if they don't exist
logs_dir = 'logs'
os.makedirs(logs_dir, exist_ok=True)

# Define file paths within the respective directories
results_file = os.path.join(logs_dir, f'model_results_{timestamp}.txt')

# Save the results to a TXT file
with open(results_file, 'w') as file:
    file.write(f"R2: {results['R2']}\n")
    file.write(f"RMSE: {results['RMSE']}\n")
    file.write("Best Parameters:\n")
    for param, value in results['Best Parameters'].items():
        file.write(f"  {param}: {value}\n")

print(f"Results saved to {results_file}")

Results saved to logs/model_results_20240918_210029.txt


In [9]:
# Display evaluation metrics
print(f"Final Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")

Final Model Performance - R^2 Score: 0.9968, RMSE: 2981.5294


In [10]:
# Define models directory, and create them if they don't exist
models_dir = 'models'
os.makedirs(models_dir, exist_ok=True)

# Train the final model using the full training+validation+test set with the optimal n_estimators
final_model = XGBRegressor(**best_params, objective='reg:squarederror', random_state=42, n_jobs=n_jobs)

# Train the model on the entire training+validation+set data
final_model.fit(X, y, verbose=100)

print("Final model trained successfully using all available data.")

Final model trained successfully using all available data.


In [11]:
# Define file paths within the respective directories
model_file = os.path.join(models_dir, f'trained_model_{timestamp}.joblib')

# Save the trained model to a file in the models folder
joblib.dump(final_model, model_file)
print(f"Model saved to {model_file}")

Model saved to models/trained_model_20240918_210029.joblib


In [12]:
# Use the final model to predict the `tow` for the submission_set_updated
submission_set_features = submission_set_updated.iloc[:,:-1]
submission_set['tow'] = final_model.predict(submission_set_features)

submission_set

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
0,248753821,2022-01-01,3b3de0f3ad0ee192513995c02f7bf7cf,LTFJ,Istanbul Sabiha Gokcen,TR,LFLL,Lyon,FR,2022-01-01T09:44:00Z,2022-01-01T12:48:33Z,B738,M,6351ec1b849adacc0cbb3b1313d8d39b,170,15,1122,69072.593750
1,248753822,2022-01-01,e06dd03d4a879ca37d9e18c1bd7cad16,EBBR,Brussels,BE,KJFK,New York JFK,US,2022-01-01T09:45:00Z,2022-01-01T17:49:51Z,A333,H,bdeeef3a675587d530de70a25d7118d2,470,15,3205,215474.828125
2,248754498,2022-01-01,2d3b1c962c78c4ebeef11bcd51b9e94c,KMIA,Miami,US,EGLL,London Heathrow,GB,2022-01-01T01:52:00Z,2022-01-01T09:55:16Z,B77W,H,5543e4dc327359ffaf5b9c0e6faaf0e1,473,10,3965,223672.390625
3,248757623,2022-01-01,81564432d3ee97c4bdf4cd8f006753dc,EGCN,Doncaster Sheffield,GB,LEAL,Alicante,ES,2022-01-01T08:20:00Z,2022-01-01T11:06:08Z,B38M,M,3922524069809ac4326134429751e26f,156,10,986,63226.054688
4,248763603,2022-01-01,84be079d7e660db105d91f600b4b3d59,EIDW,Dublin,IE,LFLL,Lyon,FR,2022-01-01T11:01:00Z,2022-01-01T13:00:43Z,A320,M,a73f82288988b79be490c6322f4c32ed,105,15,686,63306.199219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105954,258066302,2022-12-31,2d3b4446c4d05a25196a9d52cab936fb,LTFJ,Istanbul Sabiha Gokcen,TR,EKCH,Copenhagen,DK,2022-12-31T09:36:00Z,2022-12-31T13:12:17Z,B38M,M,6351ec1b849adacc0cbb3b1313d8d39b,201,15,1199,67619.484375
105955,258068609,2022-12-31,253fd692ed441fac523081471c067772,LOWW,Vienna,AT,KIAD,Washington Dulles,US,2022-12-31T09:49:00Z,2022-12-31T19:38:26Z,B763,H,5d407cb11cc29578cc3e292e743f5393,575,14,3937,175100.078125
105956,258068876,2022-12-31,c9fca302ca2e28acab0eb0bb1b46f11b,LTFM,iGA Istanbul,TR,LSZH,Zurich,CH,2022-12-31T09:25:00Z,2022-12-31T12:24:24Z,A321,M,6351ec1b849adacc0cbb3b1313d8d39b,154,25,988,74976.968750
105957,258064675,2022-12-31,00f96ad0e382476649574ba044c764fc,EHAM,Amsterdam,NL,EDDF,Frankfurt,DE,2022-12-31T10:04:21Z,2022-12-31T10:55:35Z,A320,M,f502877cab405652cf0dd70c2213e730,42,9,240,60707.332031


In [13]:
# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")
submission_set.to_csv(submission_file, index=False)